<h1><b>Задача по предсказанию рейтинга шоколада

<h3><b>Никнейм на Kaggle: 

# Описание задачи

In [741]:
# Даны характеристики шоколадок и по ним нужно предсказать их рейтинг.

# Описание данных
# Company - компания производитель
# Specific Bean Origin - географический регион происхождения
# REF - параметр, определяющий момент внесения записи в базу данных (чем выше значение, тем "свежее" запись)
# Review - дата публикации отзыва
# Cocoa Percent - процентное содержание какао
# Company Location - страна производитель
# Rating - экспертный рейтинг
# Bean Type - используемый сорт какао-бобов, если таковой имеется
# Broad Bean Origin - географический регион происхождения какао-бобов

# Файлы с данными
# choco_train.csv - тренировочные данные
# choco_test_new.csv - тестовые данные
# choco_sample_submission.csv - пример submission

# Импорт библиотек, константы

In [742]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [743]:
import warnings
warnings.filterwarnings("ignore")

In [744]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler, PolynomialFeatures
from category_encoders.ordinal import OrdinalEncoder
from category_encoders.one_hot import OneHotEncoder
from category_encoders.target_encoder import TargetEncoder
from category_encoders.leave_one_out import LeaveOneOutEncoder
from sklearn.preprocessing import LabelEncoder

In [745]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostRegressor, CatBoostClassifier
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import StackingRegressor
import optuna

In [746]:
from scipy import stats
from association_metrics import CramersV
import phik
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import r2_score
from sklearn.pipeline import Pipeline
import pickle

In [747]:
SEED = 4784

<!-- # Загрузка данных -->

# Загрузка данных

In [748]:
# TRAIN = "https://raw.githubusercontent.com/evgpat/edu_stepik_practical_ml/main/datasets/chocolate_train.csv"
# TEST = "https://raw.githubusercontent.com/evgpat/edu_stepik_practical_ml/main/datasets/chocolate_test_new.csv"
TRAIN = 'data\chocolate-rating\chocolate_train.csv'
TEST = 'data\chocolate-rating\chocolate_test_new.csv'

In [749]:
train = pd.read_csv(TRAIN)

In [750]:
test = pd.read_csv(TEST)

# Обзор данных

Описание данных
* Company - компания производитель
* Specific Bean Origin - географический регион происхождения
* REF - параметр, определяющий момент внесения записи в базу данных (чем выше значение, тем "свежее" запись)
* Review - дата публикации отзыва
* Cocoa Percent - процентное содержание какао
* Company Location - страна производитель
* Rating - экспертный рейтинг
* Bean Type - используемый сорт какао-бобов, если таковой имеется
* Broad Bean Origin - географический регион происхождения какао-бобов

In [751]:
train = train.replace('\xa0', np.NaN)
test = test.replace('\xa0', np.NaN)

Рассмотрим на первые 5 строчек, чтобы понять, как выглядят значения.

In [752]:
train = train.fillna('missing')
test = test.fillna('missing')

In [753]:
train.head()

,Company,Specific Bean Origin,REF,Review,Cocoa Percent,Company Location,Rating,Bean Type,Broad Bean Origin
0,Willie's Cacao,Rio Caribe,457,2009,72%,U.K.,3.25,Trinitario,Venezuela
1,Beschle (Felchlin),"Ocumare, Premier Cru, Quizas No. 2",508,2010,72%,Switzerland,3.50,missing,Venezuela
2,Dark Forest,Tanzania,1554,2015,70%,U.S.A.,3.00,missing,Tanzania
3,Brasstown aka It's Chocolate,Cooproagro,1125,2013,72%,U.S.A.,3.00,Trinitario,Dominican Republic
4,Pralus,"Java, Indonesie",32,2006,75%,France,3.50,Criollo,Indonesia


In [754]:
X = train.drop(columns=['Rating'])
y = train['Rating']

In [755]:
cat_feathures = train.select_dtypes(exclude='number').columns.tolist()

In [756]:
X[cat_feathures] = X[cat_feathures].apply(lambda x: x.str.strip())

In [757]:
cb = CatBoostRegressor(cat_features=cat_feathures, silent=True)

In [758]:
# cb.fit(X, y)

In [759]:
# lbm = LGBMRegressor()

# # One-hot кодирование
# ohe = OneHotEncoder(use_cat_names=True)
# X_encoded = ohe.fit_transform(X[cat_feathures])

# # Создание DataFrame с уникальными именами столбцов
# X_lbm = pd.DataFrame(
#     X_encoded,
#     columns=ohe.get_feature_names_out(cat_feathures),
#     index=X.index
# ).astype(np.int32)

In [760]:
# # Заменяем ВСЕ недопустимые символы на '_'
# X_lbm.columns = X_lbm.columns.str.replace(r'[^\w]', '_', regex=True)


# # Дополнительно убираем двойные подчёркивания (если появились)
# X_lbm.columns = X_lbm.columns.str.replace(r'__+', '_', regex=True)

# # Убираем подчёркивания в начале/конце имён (опционально)
# X_lbm.columns = X_lbm.columns.str.strip('_')

In [761]:
# X_lbm.columns = X_lbm.columns.str.replace(r'[^a-zA-Z0-9_]', '_', regex=True)
# X_lbm.columns = X_lbm.columns.str.replace(r'__+', '_', regex=True).str.strip('_')

In [762]:
# # Очистка имён столбцов
# X_lbm.columns = X_lbm.columns.str.replace(r'[^\w]', '_', regex=True)  # Всё, кроме \w, заменяем на _
# X_lbm.columns = X_lbm.columns.str.replace(r'__+', '_', regex=True)  # Убираем дубли
# X_lbm.columns = X_lbm.columns.str.strip('_')  # Убираем _ в начале/конце

In [763]:
# lbm.fit(X_lbm, y)

In [764]:
# print("Количество столбцов:", X_lbm.shape[1])
# print("Количество уникальных имён:", X_lbm.columns.nunique())

# # Вывести дублирующиеся имена
# duplicate_cols = X_lbm.columns[X_lbm.columns.duplicated()].tolist()
# print("Дублирующиеся столбцы:", duplicate_cols)

In [767]:
xgb = XGBRegressor(enable_categorical=True)

In [768]:
xgb.fit(X, y)

ValueError: DataFrame.dtypes for data must be int, float, bool or category. When categorical type is supplied, the experimental DMatrix parameter`enable_categorical` must be set to `True`.  Invalid columns:Company: object, Specific Bean Origin: object, Cocoa Percent: object, Company Location: object, Bean Type: object, Broad Bean Origin: object

In [ ]:
stop

NameError: name 'stop' is not defined

# Сравнение результатов

In [769]:
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, root_mean_squared_error, mean_squared_error

In [770]:
submission_32 = pd.read_csv('choko_submission_32.csv', usecols= ['Rating'])
submission_32 = submission_32.values

In [771]:
submission_35 = pd.read_csv('choko_submission_35.csv', usecols= ['Rating'])
submission_35 = submission_35.values

In [772]:
submission_35_36 = pd.read_csv('choko_submission_35_36.csv', usecols= ['Rating'])
submission_35_36 = submission_35_36.values

In [773]:
submission_35_32 = pd.read_csv('choko_submission_35_32.csv', usecols= ['Rating'])
submission_35_32 = submission_35_32.values

In [774]:
submission_37 = pd.read_csv('choko_submission_37.csv', usecols= ['Rating'])
submission_37 = submission_37.values

In [775]:
submission_102 = pd.read_csv('choko_submission_102.csv', usecols= ['Rating'])
submission_102 = submission_102.values

In [776]:
submission_105 = pd.read_csv('choko_submission_105.csv', usecols= ['Rating'])
submission_105 = submission_105.values

In [777]:
submission_111 = pd.read_csv('choko_submission_111.csv', usecols= ['Rating'])
submission_111 = submission_111.values

In [778]:
submission = pd.read_csv('submission.csv', usecols= ['Rating'])
submission = submission.values

In [779]:
subs_report = pd.DataFrame()

submission_lst = [(submission_32, '32'), (submission_35, '35'), (submission_35_36, '3536'), (submission_35_32, '3532'), 
                  (submission_37, '37'), (submission_102, '102'), (submission_105, '105'), (submission_111, '111'), (submission, 'submission')]

for i in range(len(submission_lst)):
    for j in range(len(submission_lst)):
        if i != j:
            i_name, j_name = submission_lst[i][1], submission_lst[j][1]
            mae = mean_absolute_error(submission_lst[i][0], submission_lst[j][0])
            mape = mean_absolute_percentage_error(submission_lst[i][0], submission_lst[j][0])
            rmse = root_mean_squared_error(submission_lst[i][0], submission_lst[j][0])
            mse = mean_squared_error(submission_lst[i][0], submission_lst[j][0])
            report = pd.DataFrame(columns=['MAE', 'MAPE', 'RMSE', 'MSE'], data=[[0, 0, 0, 0]])
            report[['MAE', 'MAPE', 'RMSE', 'MSE']] =  mae, mape, rmse, mse
            report.index = [f'{i_name}-{j_name}',]
            subs_report = pd.concat([subs_report, report], axis=0)

    # return subs_report



# Tail